In [68]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from konlpy.tag import Okt
import timeit
import json
okt = Okt()

In [47]:
df = pd.read_csv("../Data/news_data/search_final_data.csv", index_col=0)
df.head(10)

,title,contents
0,"[특징주]""고마워요, 법무부 장관님""...'윤석열 테마주' 초강세",‘윤석열 테마주’로 알려진 서연(007860)이 강세다. 이날 오후 1시 34분 기...
1,"‘방탄소년단 테마주’ 모두 하락, 엔터테인먼트3사는 YG만 올라",'방탄소년단 테마주'로 묶이는 회사 주가가 모두 내렸다. 25일 키이스트 주가는 전...
2,주가 뜬 테마주 매도한 내부자들...하락 몫은 개인투자자,테마주로 엮이면서 주가가 급등한 상장사의 최대주주나 특수관계인들이 지분을 팔아치우는...
3,'수소트럭' 美니콜라 주가 급등···국내 수소테마주도 '고공행진',두산퓨얼셀이나 에스퓨얼셀 등 연료전지 업체는 물론 니콜라에 투자한 한화그룹까지 '수...
4,테마주 취급받는 우선주…'폭탄 돌리기' 현재진행형,일부 우선주가 급등 이후 급락을 반복하는 롤러코스터를 타는 가운데 시장에서는 '우선...
5,"'코로나19 테마주' 한국유나이티드제약, 종식 위한 사회적노력 '역행' 논란","한편, 한국유나이티드제약은 이른바 '코로나19 테마주' 중 하나로 꼽힌다. 코로나1..."
6,가전·맥주·빙과…여름 테마주 웃을까,올 여름철 역대급 무더위가 찾아올 것으로 예상되면서 관련주들이 주목받고 있다. 24...
7,"'방탄소년단 테마주' 대부분 올라, 엔터테인먼트3사는 JYP 빼고 상승",'방탄소년단 테마주'로 묶이는 회사 주가가 대부분 올랐다. 23일 초록뱀미디어 주가...
8,쌍방울·남영비비안 마스크 허위 계약 논란...일방적 계약 파기 VS 일시적 주...,당시 주가를 살펴보면 마스크 테마주에 편입된 남영비비안은 28일 전 거래일 종가 대...
9,"[특징주] '이낙연 테마주' 남선알미늄, 2.42% 상승",알루미늄 등 비철금속을 제조·판매하는 남선알미늄은 계열 관계인 SM그룹 삼환기업 전...


In [48]:
kospi = pd.read_csv("../Data/stock_data/kospi.csv", index_col=0, encoding='cp949')
kosdaq = pd.read_csv("../Data/stock_data/kosdaq.csv", index_col=0, encoding='cp949')

In [49]:
company_list = pd.concat([kospi[['종목코드', '기업명']], kosdaq[['종목코드', '기업명']]], axis=0)
company_list = company_list.sort_values('기업명').reset_index().drop(columns='번호')
company_list.head(5)

,종목코드,기업명
0,60310,3S
1,95570,AJ네트웍스
2,6840,AK홀딩스
3,54620,APS홀딩스
4,265520,AP시스템


In [50]:
def delete_one_letter(x: str):

    # Delete one letter
    temp = list(filter(lambda i: (len(i)>1), x))
    x = temp
    
    return x

In [52]:
# Extract theme from title
def extract_theme(x: list):
    
    theme_words = ['테마', '테마주', '관련', '관련주']
    result = None
    for i in range(len(x)):
        # Handle with XX 테마주
        if x[i] in theme_words:
            result = x[i-1]
        # Handle with XX테마주
        elif any(theme_word in x[i] for theme_word in theme_words):
            for theme_word in theme_words:
                if theme_word in x[i]:
                     result = re.sub(theme_word, '', x[i])
    
    return result

In [53]:
# Extract comapny name from contents
def extract_company(x: list):
    
    result = []
    [result.append(i) for i in x if i in(list(company_list['기업명']))]
    return result

In [54]:
start_time = timeit.default_timer()

df['title'] = df['title'].apply(lambda x: re.sub('[^a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', x))
df['contents'] = df['contents'].apply(lambda x: re.sub('[^a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', x))

# Extract only nouns and split
df['title'] = df['title'].apply(lambda x: okt.nouns(x))
df['contents'] = df['contents'].apply(lambda x: x.split())

# Delete one letter
df['title'] = df['title'].apply(delete_one_letter)
df['contents'] = df['contents'].apply(delete_one_letter)

# Extract theme and company
df['title'] = df['title'].apply(extract_theme)
df['contents'] = df['contents'].apply(extract_company)

terminate_time = timeit.default_timer()
print("%f초 걸렸습니다." % (terminate_time - start_time))

328.405419초 걸렸습니다.


In [60]:
# Drop empty rows
drop_index = []
for i in range(len(df)):
    if (df['title'][i] == None) or (df['contents'][i] == []):
        drop_index.append(i)

df.drop(index=drop_index, inplace=True)
df.reset_index(inplace=True, drop=True)

In [63]:
dic = {}
for i in tqdm(range(len(df))):
    if df.title[i] not in dic.keys():
        dic[df.title[i]] = df.contents[i]
    elif df.title[i] in dic.keys():
        dic[df.title[i]] += df.contents[i]
        
for i in (range(len(df))):
    dic[df.title[i]] = list(set(dic[df.title[i]]))

100%|██████████| 27321/27321 [00:02<00:00, 13429.29it/s]


In [72]:
df_final = pd.DataFrame(dic.items(), columns=['theme', 'company'])

In [74]:
df_final

,theme,company
0,윤석열,"[서연, 안랩, 노루페인트, 오공, 웰크론, 케이엠, 화천기계, 모베이스전자, 모베..."
1,방탄소년단,"[엘비세미콘, 키이스트, 초록뱀, 디피씨, 손오공, 드림어스컴퍼니, 경남제약, 남성..."
2,국내,"[에스퓨얼셀, 포스코케미칼, 일진머티리얼즈, 국제약품, 두산퓨얼셀, 웰크론, 우리기..."
3,애국,"[경인양행, 한화, 후성, SK하이닉스, 제이에스티나, 대신증권, 토니모리, 남영비..."
4,보톡스,"[대웅제약, 휴온스글로벌, 제테마, 메디톡스, 휴젤]"
...,...,...
721,약주,"[녹십자, 유한양행, 대한뉴팜, 한미약품, 삼성바이오로직스]"
722,본격,[아시아나항공]
723,에어부산,[아시아나IDT]
724,중동,[현대자동차]


In [75]:
df.to_csv("../Data/data_for_use_v2.csv")